In [1]:
from FasterRCNN_predict import FasterRCNN_predict
from PIL import Image
from tqdm.notebook import tqdm
from torchvision.ops import box_iou
from torch import tensor
import numpy as np
import os

In [1]:
from image_intensification import ImageIntensification
with open("./train/val.txt")as f:
    ii_lines = f.readlines()
image_intensification = ImageIntensification()
image_intensification(ii_lines, 10)

## 保存预测结果

In [2]:
val_data_save_path = './val_data_predict.txt'
# train_data_save_path = './train_data_predict.txt'
# intensified_data_save_path = './intensified_data_predict.txt'

In [3]:
with open("./train/val.txt") as fval:
    val_data_bace = tuple(fval.readlines())
# with open("./train/train.txt") as ftrain:
#     train_data_bace = tuple(ftrain.readlines())
# with open("./intensified_image.txt") as fintensified:
#     intensified_data_bace = tuple(fintensified.readlines())

In [4]:
def store_forecast_results(data_bace, model, store_dir):
    with open(store_dir, 'w') as f:
        for line in tqdm(data_bace):
            datas = line.replace('\n','').split(' ')
            image = Image.open(datas[0])
            bbox = model.detect_coordinates(image)
            data_str = ''
            for i in range(len(bbox[0])):
                data_str += str(bbox[0][i]) + ',' + str(bbox[1][i]) + ',' + str(bbox[2][i]) + ',' + str(bbox[3][i]) + ',' + str(bbox[4][i])
                if i != len(bbox[0]) - 1:
                    data_str += ' '
            if data_str == '':
                data_str += 'x|'
            else:
                data_str += '|'
            if len(datas) == 2:
                data_str += datas[-1] + '\n'
            else:
                data_str += '\n'
            f.write(data_str)

In [5]:
model = FasterRCNN_predict()

model_data/weights.pth model, anchors, and classes loaded.


In [6]:
store_forecast_results(val_data_bace, model, val_data_save_path)

  0%|          | 0/624 [00:00<?, ?it/s]

## 解析预测结果

In [18]:
def print_result(TP, TN, FP, FN):
    print("准确率acc ..............%.2f%%"%((TP+TN)/(TP+TN+FP+FN)*100))
    print("灵敏度sens .............%.2f%%"%(TP/(TP+FN)*100))
    print("特异度(真负例率)spec ...%.2f%%"%(TN/(TN+FP)*100))
    print("阳性预测率(精准度)ppv ..%.2f%%"%(TP/(TP+FP)*100))
    print("阴性预测率npv ..........%.2f%%"%(TN/(FN+TN)*100))
    print("F1 score ...............%.2f"%(2*TP/(2*TP+FP+FN)))

In [7]:
def write_roc_data(TP, TN, FP, FN, confidence, iou):
    dir = r"C:\Users\AbaKir\Desktop\roc_data.csv"
    if not os.path.exists(dir):
        with open(dir, 'w')as f:
            f.write("confidence,TP,TN,FP,FN,准确率,灵敏度,特异度,阳性预测率,阴性预测率,F1得分,IoU\n")
#             f.write("confidence,TP,TN,FP,FN,准确率,灵敏度,阳性预测率,F1得分,IoU\n")
    f = open(dir, 'a')
    f.write(str(confidence) + ",")
    f.write(",".join(map(str,[TP, TN, FP, FN])))
    f.write("," + str((TP+TN)/(TP+TN+FP+FN)))
    f.write("," + str(TP/(TP+FN)))
    f.write("," + str(TN/(TN+FP)))  #
    f.write("," + str(TP/(TP+FP)))
    f.write("," + str(TN/(FN+TN)))  #
    f.write("," + str(2*TP/(2*TP+FP+FN)))
    f.write("," + str(iou))
    f.write("\n")
    f.close()

In [8]:
with open(val_data_save_path, 'r')as f:
    val_data_predict = tuple(f.readlines())
for confidence in np.array(range(50, 100, 5))/100:
    TP, TN, FP, FN = 0, 0, 0 ,0
    ious, num_iou = 0., 0
    for line in val_data_predict:
        data = line.split('|')
        if data[0] == 'x' and data[1] == '\n': 
            TN += 1
        elif data[0] == 'x' and data[1] != '\n': 
            FN += 1
        elif data[0] != 'x' and data[1] == '\n':
            FP += 1
        else:
            flag = True
            for predict in data[0].split(' '):
                if float(predict.split(',')[-1]) < confidence:
                    continue
                bbox_row = list(map(int,predict.split(',')[:-1]))
                iou = box_iou(tensor([bbox_row]),
                              tensor([list(map(int,data[1].split(',')[:-1]))])).numpy()[0][0]
                ious += iou
                num_iou += 1
                if iou >= 0.5 and flag:
                    TP += 1
                    flag = False
                else: FP += 1
            if flag: FN += 1
    write_roc_data(TP, TN, FP, FN, confidence, (ious)/(num_iou))
#     print(TP, TN, FP, FN, confidence, (ious)/(num_iou))

In [32]:
print(TP, TN, FP, FN)

'4393,7986,244,4957'

In [26]:
a = tensor([[59,167,135,227]])
b = tensor([[198,92,254,190]])
box_iou(a,b)

tensor([[0.]])